In [3]:
n=30

In [4]:
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx
from collections import defaultdict
from matplotlib import colors

In [5]:
#CREATE VISUAL OF THE SIMULATED STATE
def show_heatmap(A):
    axis1 = []
    axis2 = []
    fig, ax = plt.subplots(figsize=(4,4))
    norm = plt.Normalize(30,70)
    im, cbar = heatmap(A, axis1, axis2, ax=ax,cmap="PuOr", norm=norm)
    fig.tight_layout()
    plt.show()
    
# NORMALIZE IMAGE MAP TO BETWEEN 10 AND 90 PERCENT PER PRECINCT
def normalize_array(C):
    return np.clip((C-10)/80, 0, 1)


def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    #ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar

def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

def make_square(s):
    square = []
    
    for q in range(s):
        square.append([])
        for r in range(s):
            square[q].append(int(np.round(mean+std*np.random.randn(1))))
    
    return square

def diff_from_nbrs(A, a, b):
    
    s = A.shape[0]
    if not ((a in [0,s-1]) or (b in [0,s-1])):
        return abs((A[a-1][b]+A[a][b-1]+A[a][b+1]+A[a+1][b])/4 - A[a][b])
    else:
        if a == 0:
            if b == 0:
                return abs((A[a+1][b]+A[a][b+1])/2-A[a][b])
            else:
                if b == s-1:
                    return abs((A[a+1][b] + A[a][b-1])/2 - A[a][b])
                else:
                    return abs((A[a+1][b] + A[a][b-1] + A[a][b+1])/3 - A[a][b])
        else:
            if a == s-1:
                if b == 0:
                    return abs((A[a-1][b]+A[a][b+1])/2-A[a][b])
                else:
                    if b == s-1:
                        return abs((A[a-1][b] + A[a][b-1])/2 - A[a][b])
                    else:
                        return abs((A[a-1][b] + A[a][b-1] + A[a][b+1])/3 - A[a][b])
            else:
                if b == 0:
                    return abs((A[a-1][b]+A[a+1][b]+A[a][b+1])/3 - A[a][b])
                else:
                    return abs((A[a-1][b]+A[a+1][b]+A[a][b-1])/3 - A[a][b])
                
def swap(A):
    s = A.shape[0]
    a = random.randint(0,s-1)
    b = random.randint(0,s-1)
    
    c = random.randint(-1,1)
    
    if c == -1 or c == 1:
        d = 0

    if c == 0:
        options = [-1,1]
        d = random.choice(options)
       
    #should we swap a,b with a+c,b+d
    try:
        diff1 = diff_from_nbrs(A,a,b)
        B = np.copy(A)
        val1        = B[a][b]
        val2        = B[a+c][b+d]
        B[a][b]     = val2
        B[a+c][b+d] = val1
        diff2 = diff_from_nbrs(B,a,b)
        if diff1 > diff2:
            val1        = A[a][b]
            val2        = A[a+c][b+d]
            A[a][b]     = val2
            A[a+c][b+d] = val1
            #print(a,b)
            return A
        return A
        
    except:
        return A

In [21]:
#total number of precincts
l =[i for i in range(n**2)]


def pick_new_node(ours, theirs, low_high, counter):
    possible = set(l).difference(set(ours)).difference(set(theirs))
    if len(possible) == 0:
        return -1
    if counter%2 ==0:
        node     = choose_according_to_strategy(list(possible), ours, theirs, counter, strategy=strategyA, low_high = low_high)
    else:
        node = choose_according_to_strategy(list(possible), ours, theirs, counter, strategy=strategyB, low_high = low_high)
    return node


def sort_high_low(leftovers,low_high):
    '''orders leftovers in ascending or descending order'''
    dic = {i:B[i] for i in leftovers}
    if low_high == "high":
        dic_sorted = {k:v for k,v in sorted(dic.items(),key=lambda item: -item[1])}
    if low_high == "low":
        dic_sorted = {k:v for k,v in sorted(dic.items(),key=lambda item: item[1])}
    return list(dic_sorted.keys())

def choose_according_to_strategy(possible_nodes, current_nodes, their_nodes, counter, strategy="random", low_high = "low"):
    '''picks a node from the unclaimed nodes according to strategy.  checks if it creates valid districts'''
    if strategy == "random":
        while True: 
            node        = random.choice(possible_nodes)
            x           = []
            x.append(node)
            nodes       = current_nodes +  x
            copy_nodes = possible_nodes.copy()
            copy_nodes.remove(node)
            if is_valid(nodes, their_nodes,copy_nodes,0):
                #print(nodes)
                #print(copy_nodes)
                #print('--------------')
                return node
    if strategy == "greedy":
        while True: 
            sorted_leftovers = sort_high_low(possible_nodes, low_high)
            for node in sorted_leftovers:
                nodes, copy_nodes = new_districts(current_nodes, node, possible_nodes)
                if is_valid(nodes, their_nodes, copy_nodes, counter):
                    return node
            #print('non-compactness encountered')
            #get here only if no compact choice exists
            sorted_leftovers        = sort_by_compactness(current_nodes, possible_nodes)
            for node in sorted_leftovers:
                nodes, copy_nodes = new_districts(current_nodes, node, possible_nodes)
                if is_valid(nodes, their_nodes, copy_nodes, 0):
                    return node
    if strategy == "barely":
        while True: 
            #figure out current district score
            num_precincts  = len(current_nodes)
            if num_precincts == 0:
                district_score = 50
            else:
                district_score = np.mean(np.array([B[node] for node in current_nodes]))
                print('\r working on precinct: {} district_score: {:.3f} '.format(counter, district_score), end = '')
                    
            sorted_leftovers = sort_barely(possible_nodes, district_score, low_high, num_precincts)
                
            for node in sorted_leftovers:
                nodes, copy_nodes = new_districts(current_nodes, node, possible_nodes)
                if is_valid(nodes, their_nodes, copy_nodes, counter):
                    return node
            #print('non-compactness encountered')
            #get here only if no compact choice exists
            sorted_leftovers        = sort_by_compactness(current_nodes, possible_nodes)
            for node in sorted_leftovers:
                nodes, copy_nodes = new_districts(current_nodes, node, possible_nodes)
                if is_valid(nodes, their_nodes, copy_nodes, 0):
                    return node

                
def sort_barely(possible_nodes, district_score, low_high, num_precincts):
    '''sort based on relation to cutoff--used for 'barely' strategy'''
    
    dic = {i:B[i] for i in possible_nodes}
    if low_high == 'high':
        x = barely_cutoff * (num_precincts +1) - num_precincts * district_score
        #now sort possible_nodes in increasing order of preference, first those above x, then below
        d2 = {k:v for k,v in dic.items() if v>=x}
        d3 = {k:v for k,v in dic.items() if v<x}
        d3_sorted = {k:v for k,v in sorted(d3.items(),key=lambda item: -item[1])}
        d2_sorted = {k:v for k,v in sorted(d2.items(),key=lambda item: item[1])}
        return list(d2_sorted.keys())+list(d3_sorted.keys())
    if low_high == 'low':
        x = (100 - barely_cutoff) * (num_precincts +1) - num_precincts * district_score
        #now sort possible_nodes in increasing order of preference, first those below x, then above
        d2 = {k:v for k,v in dic.items() if v<=x}
        d3 = {k:v for k,v in dic.items() if v>x}
        d2_sorted = {k:v for k,v in sorted(d2.items(),key=lambda item: -item[1])}
        d3_sorted = {k:v for k,v in sorted(d3.items(),key=lambda item: item[1])}
        return list(d2_sorted.keys())+list(d3_sorted.keys())
    
    
    
def sort_by_compactness(nodes,leftovers):
    '''sorts leftovers by how compact nodes would become'''
    scores = [compute_compactness(add_node(nodes,node)) for node in leftovers]
    #print(scores)
    dic = {leftovers[i]:scores[i] for i in range(len(leftovers))}
    dic_sorted = {k:v for k,v in sorted(dic.items(),key=lambda item: -item[1])}
    #print(dic_sorted)
    return list(dic_sorted.keys())
    
    

def add_node(nodes,node):
    x = []
    x.append(node)
    new_nodes = nodes + x
    return new_nodes

def new_districts(nodes, node, leftovers):
    x           = []
    x.append(node)
    new_nodes       = nodes +  x
    copy_nodes = leftovers.copy()
    copy_nodes.remove(node)
    return new_nodes, copy_nodes
    
def check_connected(nodes):
    Q = G.subgraph(nodes)
    if len(nodes)==0 or nx.is_connected(Q):
        return True
    return False

def check_district(nodes):
    Q = G.subgraph(nodes)
    if len(nodes)==0 or (nx.is_connected(Q) and check_compact(Q)):
        return True
    return False

def check_compact(nodes):
    return compute_compactness(nodes) > compact_threshold

def compute_compactness(nodes, boundaries = True):
    '''computes compactness score.  if boundaries = True, adds one edge to each boundary vertex
    this should inhibit the growth of ---boundary tendrils----'''
    Q = G.subgraph(nodes)
    if boundaries:
        #compute number of boundary vertices
        num_nodes = 0
        for  i in nodes:
            if int(i/n)  in [0,n-1]:
                num_nodes += 1
            if i%n in [0,n-1]:
                num_nodes += 1
        return (2*Q.number_of_edges()+num_nodes)/float(Q.number_of_nodes())
    else:
        return 2*Q.number_of_edges()/float(Q.number_of_nodes())

def is_valid(nodes, other_nodes, leftovers, counter):
    '''check whether all leftover points are reachable from both districts'''
    if counter < burn_in:
        return check_connected(nodes) and check_connected(nodes+leftovers) and check_connected(other_nodes+leftovers)  
    return check_district(nodes) and check_connected(nodes+leftovers) and check_connected(other_nodes+leftovers)

burn_in = 10 #number of turns before start of enforcing compactness
#compact_threshold = 3.75 #average degree necessary to count as compact
barely_cutoff = 51 #minimum score desired in 'barely' startegy for majority group


aa = []
bb = []
def play_game():
    a = []
    b = []
    counter = 0
    while counter < 2000:
        #if counter%100 ==0:
            #print('working on precinct: {}'.format(counter))
        x = pick_new_node(a,b,"high", counter)
        if x < 0:
            print('done!')
            break
        else:
            a.append(x)
            c = a.copy()
            aa.append(c)
            counter += 1
            
        x = pick_new_node(b,a,"low", counter)
        if x < 0:
            print('done!')
            break
        else:
            b.append(x)
            d = b.copy()
            bb.append(d)
        counter += 1
   # print(a)
   # print(b)
    return a,b

def compute_district_stats(L):
    '''computes some stats about a formed district'''
    #compute average score for precincts in district
    scores = {node:B[node] for node in list(L.nodes)}
    return np.mean(np.array(list(scores.values()))), compute_compactness(L.nodes(), boundaries = True)

In [7]:
def grid_value(x):
    '''finds grid coordinate of an integer'''
    row = int(x/n)
    col = x % n
    return row,col
def plot_graph(xs,ys,i, strategyA, strategyB, compact_threshold, mean):
    C = np.zeros((n,n))
    for x in xs:
        C[grid_value(x)] = 1.0
    for y in ys:
        C[grid_value(y)] = 2.0
    cmap = colors.ListedColormap(['white','black','white'])
    bounds = [0,1,2,3]
    norm   = colors.BoundaryNorm(bounds,cmap.N)
    fig,ax = plt.subplots(figsize=(10,10))
    plt.tick_params(left=False,
                bottom=False,
                labelleft=False,
                labelbottom=False)
    ax.axis('off')
    f1 = plt.imshow(C,cmap=cmap,norm=norm, alpha = 0.99)
    f2 = plt.imshow(normalize_array(A), cmap="PuOr",alpha=0.6)
    filename2 = 'imagecomplete'+strategyA+'_'+strategyB+'_'+str(mean) + '_' + str(i) + '_' + str(compact_threshold) + '.png'
    plt.savefig(filename2)
    plt.close('all')

In [27]:
#test cell for experiment 2/1/23

for M in [108, 110, 112]:
    m=M/2
    compact_threshold = 0.0
    mean = m
    results = defaultdict(list)
    for t in range(6):
        print('working on iteration {}'.format(t))
        n = 30
        std = 10
        A = np.array(make_square(n))
        print(A[1][1])
        strats = ['barely', 'greedy', 'random']
        for _ in range(1000000):
                    swap(A)
        print(A[1][1])
        for strategyA in strats:
            for strategyB in strats:
                print('working on {}{}'.format(strategyA,strategyB))
                scores = []

                B = A.reshape(n**2)
        
                G = nx.Graph()
                G.positions = {i:(int(i/n),i%n) for i in range(n**2)}
                for i in range(n**2):
                    if not i%n==n-1:
                        G.add_edge(i,i+1)
                    if not int(i/n)==n-1:
                        G.add_edge(i,i+n)
                labels = {}
                j = 0
                for node in list(G.nodes()): 
                    labels[node] = B[node]
         
                district_a, district_b = play_game()  
                K1 = G.subgraph(district_a)
                score1,_ = compute_district_stats(K1)
   
                K2 = G.subgraph(district_b)
                score2,_ = compute_district_stats(K2)

                scores.append((score1,score2))
                plot_graph(district_a,district_b,t, strategyA, strategyB, compact_threshold, m)
                results[strategyA+strategyB].append(scores)
    filename = 'complete_scores_'+str(mean) + '_' + str(compact_threshold)   
    np.save(filename,results)

working on iteration 0
46
61
working on barelybarely
 working on precinct: 899 district_score: 54.902 done!
working on barelygreedy
 working on precinct: 898 district_score: 54.831 done!
working on barelyrandom
 working on precinct: 898 district_score: 52.964 done!
working on greedybarely
 working on precinct: 899 district_score: 51.027 done!
working on greedygreedy
done!
working on greedyrandom
done!
working on randombarely
 working on precinct: 899 district_score: 52.548 done!
working on randomgreedy
done!
working on randomrandom
done!
working on iteration 1
75
51
working on barelybarely
 working on precinct: 899 district_score: 52.122 done!
working on barelygreedy
 working on precinct: 898 district_score: 55.114 done!
working on barelyrandom
 working on precinct: 898 district_score: 53.200 done!
working on greedybarely
 working on precinct: 899 district_score: 51.029 done!
working on greedygreedy
done!
working on greedyrandom
done!
working on randombarely
 working on precinct: 899 d